In [ ]:
from subprocess import call
import os
import glob
from multiprocessing import Process
from pathlib import Path

In [ ]:
# Download the Flickr dataset

flickr_root = Path('../data/external/flickr_images')

f = open(str(flickr_root / 'good_jpgs'))
good_jpgs = [Path(l.strip()).stem for l in f.readlines()]

prs = []
for model in [d for d in flickr_root.iterdir() if d.is_dir()]:
    # pick 'good jpgs'
    with open(str(model / 'urls_final')) as f_in:
        urls = [l.strip() for l in f_in.readlines()]
    urls_good = [url for url in urls if Path(url).stem in good_jpgs]
    with open(str(model / 'urls_good'), 'w') as f_out:
        for url in urls_good:
            f_out.write(url + '\n')

    command = 'wget -i {} -P {}'.format(str(model / 'urls_good'), str(model))
    print(command)
    p = Process(target=lambda cmd: call(cmd, shell=True), args=(command,))
    p.start()
    prs.append(p)

for p in prs:
    p.join()
    print(p.name, 'done')

In [ ]:
# Download the reviews (validation) dataset

images_root = Path('../data/external/val_images')

prs = []
for urls_file in images_root.glob('*/urls_*'):
    command = 'wget -i {} -P {}'.format(str(urls_file), str(urls_file.parent))
    print(command)
    p = Process(target=lambda cmd: call(cmd, shell=True), args=(command,))
    p.start()
    prs.append(p)

for p in prs:
    p.join()
    print(p.name, 'done')